In [1]:
#Import libraries
import pandas as pd # CSV file I/O (pd.read_csv)
import numpy as np #linear algebra
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import PolynomialFeatures
#Import data
gini = pd.read_csv('gini.csv')
prod = pd.read_csv('Productivity.csv')

In [2]:
#Cleaning the data
gini.drop(['Country Code'],axis=1,inplace=True)
gini.columns.values[0]='Country'
gini.columns.values[2]='Gini'
prod = prod.drop(['LOCATION','TIME','Subject', 'Unit','Unit Code','MEASURE','Measure','PowerCode','Reference Period Code','Reference Period', 'PowerCode Code','Flag Codes', 'Flags'], axis=1)
prod.replace("China (People's Republic of)","China",inplace=True)
gini.replace("Russian Federation","Russia",inplace=True)
gini.replace("Korea, Rep.","Korea",inplace=True)

In [3]:
#Confirming absence of null values
gini.isnull().any().sum() + prod.isnull().any().sum()

0

In [4]:
prod_gdpemp = prod[prod['SUBJECT'] == 'T_GDPEMP'] #GDP per person employed
prod_gdppop = prod[prod['SUBJECT'] == 'T_GDPPOP'] #GDP per head of population
prod_gdphrs = prod[prod['SUBJECT'] == 'T_GDPHRS'] #GDP per hour worked

In [5]:
def intersection(lst1, lst2): 
    lst3 = [value for value in lst1 if value in lst2] 
    return lst3 
gdpemp_countries = prod_gdpemp['Country'].unique().tolist()
gini_countries = gini['Country'].unique().tolist()
countries = intersection(gdpemp_countries, gini_countries)
countries.sort()
countries
#countries included in both GiniPerEmp and Productivity set

['Australia',
 'Austria',
 'Belgium',
 'Brazil',
 'Canada',
 'Chile',
 'China',
 'Colombia',
 'Czech Republic',
 'Denmark',
 'Estonia',
 'Finland',
 'France',
 'Germany',
 'Greece',
 'Hungary',
 'Iceland',
 'India',
 'Indonesia',
 'Ireland',
 'Israel',
 'Italy',
 'Japan',
 'Korea',
 'Latvia',
 'Lithuania',
 'Luxembourg',
 'Mexico',
 'Netherlands',
 'Norway',
 'Poland',
 'Portugal',
 'Russia',
 'Slovak Republic',
 'Slovenia',
 'South Africa',
 'Spain',
 'Sweden',
 'Switzerland',
 'Turkey',
 'United Kingdom',
 'United States']

In [6]:
def difference(lst1, lst2): 
    lst3 = [value for value in lst1 if value not in lst2] 
    return lst3 
def merge(list1, list2): 
    merged_list = tuple(zip(list1, list2))  
    return merged_list 

In [7]:
import warnings; warnings.simplefilter('ignore')
corr = []
impute_iter = []
iter = 10; #number of imputations
for i in range (iter): 
    for country in countries:
        pyears = prod_gdpemp[prod_gdpemp.Country == country].Time.tolist()
        pyear_min = pyears[0]
        pyear_max = pyears[len(pyears)-1]
        
        g_gdpemp_years = gini[gini.Country == country].Year.tolist()
        gyear_min = g_gdpemp_years[0]
        gyear_max = g_gdpemp_years[len(g_gdpemp_years)-1]

        if(gyear_min < pyear_min):
            minyear = pyear_min 
        else:
            minyear = gyear_min
        if(gyear_max > pyear_max):
            maxyear = pyear_max
        else:
            maxyear = gyear_max
        
        #requiring four data points for multiple imputation validity
        if(maxyear - minyear > 2):
            #years available from productivity
            prod_range = [i for i in pyears if i >= minyear and i <=maxyear]
            gini_range = [i for i in g_gdpemp_years if i >= minyear and i <=maxyear]
            gyear_lacks = difference(prod_range,gini_range)
            
            #productivity
            x = prod_gdpemp[prod_gdpemp.Country == country]
            x1 = x.loc[(x['Time'] >= minyear) & (x['Time'] <= maxyear)]
            prod_years = x1.Time.tolist()
            prod_values = x1.Value.tolist() 
            prod_full = merge(prod_years,prod_values)

            #Isolate Gini years to match productivity
            y = gini[gini.Country == country]
            y1 = y.loc[(y['Year'] >= minyear) & (y['Year'] <= maxyear)]
            gini_years = y1.Year.tolist()
            gini_values = y1.Gini.tolist() 
            noise = np.random.normal(0,np.std(gini_values),len(prod_range))

            #gini imputated with a polynomial degree 3
            g = np.polyfit(y1.Year.tolist(), y1.Gini.tolist(), 3)
            g1 = np.poly1d(g)
            gini_imp = []
            
            for i in range(len(gyear_lacks)):
                gini_imp.append(g1(gyear_lacks[i]) + noise[i])

            gini_missing = merge(gyear_lacks, gini_imp)
            gini_original = merge(gini_years, gini_values)
            gini_full = gini_original + gini_missing
            gini_full = sorted(gini_full, key=lambda tup: tup[0])

            gini_full_gini = []
            prod_full_prod = []

            for i in range(len(gini_full)):
                gini_full_gini.append(gini_full[i][1])
                prod_full_prod.append(prod_full[i][1])
                
            corr.append(np.corrcoef(prod_full_prod, gini_full_gini)[0][1])
            impute_iter.append((country, np.corrcoef(prod_full_prod, gini_full_gini)[0][1]))
            corr = []

In [8]:
print("Correlation between Gini and GDP per Employee with", iter,"imputations")
#the impute_iter contains all iterations of the imputation, performed n = iter times
avg_len = int(len(impute_iter)/(iter))
averages = [0] * avg_len

country_list = []
for i in range(0, avg_len):
    country_list.append(impute_iter[i][0])
    
for i in range(0, len(impute_iter)):
    averages[i % avg_len] = averages[i % avg_len] + impute_iter[i][1]
    
to_sort = []

for i in range(0, avg_len):
    averages[i] = averages[i]/iter
    to_sort.append((country_list[i], averages[i]))

to_sort = sorted(to_sort, key=lambda tup: tup[1])
for i in range(0, avg_len):
    print(to_sort[i])

print("Average correlation:", sum(averages)/len(averages))

Correlation between Gini and GDP per Employee with 10 imputations
('Brazil', -0.9529113205558032)
('Portugal', -0.8654758540101499)
('China', -0.7954338302741701)
('Slovenia', -0.793400494648852)
('Chile', -0.7866035592967691)
('Estonia', -0.7641520418860284)
('Switzerland', -0.7353102925135542)
('Mexico', -0.7242058730207294)
('United Kingdom', -0.7176426067387617)
('Belgium', -0.6590023024538468)
('Norway', -0.6435504418174655)
('Finland', -0.6178097806942829)
('Colombia', -0.5563494810295434)
('Turkey', -0.491131214553572)
('Netherlands', -0.45766574953770955)
('Iceland', -0.3333970573179649)
('Czech Republic', -0.25562442271431474)
('Ireland', -0.10246651242626467)
('Korea', -0.08210136676262306)
('Slovak Republic', -0.03528601860896927)
('Russia', 0.04867146573212773)
('Hungary', 0.1067201262573162)
('Latvia', 0.21158073771738234)
('Poland', 0.35443398131310283)
('Lithuania', 0.390371640884667)
('Italy', 0.3962433185172252)
('Israel', 0.4914178062209153)
('Luxembourg', 0.567951497